In [75]:
import pandas as pd

businesses = pd.read_csv("businesses.csv")
new_businesses = pd.read_csv("new_businesses.csv")
countries = pd.read_csv("countries.csv")
categories = pd.read_csv("categories.csv")

In [86]:
business_country = businesses.merge(countries, on='country_code', how='inner', validate='one_to_one')
print(f'DataFrame size: {business_country.shape}')
pd.DataFrame({'is_null' : business_country.isna().any()})

DataFrame size: (163, 6)


,is_null
business,False
year_founded,False
category_code,False
country_code,False
country,False
continent,False


In [77]:
business_country.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163 entries, 0 to 162
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   business       163 non-null    object
 1   year_founded   163 non-null    int64 
 2   category_code  163 non-null    object
 3   country_code   163 non-null    object
 4   country        163 non-null    object
 5   continent      163 non-null    object
dtypes: int64(1), object(5)
memory usage: 7.8+ KB


In [78]:
business_country.sort_values('year_founded', ascending=True, inplace=True)
oldest_business = business_country.groupby('continent')['year_founded'].min().reset_index()
oldest_business

,continent,year_founded
0,Africa,1772
1,Asia,578
2,Europe,803
3,North America,1534
4,Oceania,1809
5,South America,1565


In [79]:
oldest_business_by_continent = pd.merge(business_country.drop(columns=['category_code', 'country_code']), oldest_business, on=['continent', 'year_founded'], how='right', validate='many_to_one')
oldest_business_by_continent = oldest_business_by_continent.reindex(['continent', 'country', 'business', 'year_founded'], axis=1)
# List of Oldest Businesses still active on each Continent
oldest_business_by_continent

,continent,country,business,year_founded
0,Africa,Mauritius,Mauritius Post,1772
1,Asia,Japan,Kongō Gumi,578
2,Europe,Austria,St. Peter Stifts Kulinarium,803
3,North America,Mexico,La Casa de Moneda de México,1534
4,Oceania,Australia,Australia Post,1809
5,South America,Peru,Casa Nacional de Moneda,1565


In [80]:
mixed_business = pd.concat([businesses, new_businesses], axis=0, ignore_index=False, keys=['old', 'new'], join='inner', verify_integrity=True)
mixed_business.reset_index(level=0, inplace=True)
mixed_business.rename(columns={'level_0': 'status'}, inplace=True)
mixed_business

,status,business,year_founded,category_code,country_code
0,old,Hamoud Boualem,1878,CAT11,DZA
1,old,Communauté Électrique du Bénin,1968,CAT10,BEN
2,old,Botswana Meat Commission,1965,CAT1,BWA
3,old,Air Burkina,1967,CAT2,BFA
4,old,Brarudi,1955,CAT9,BDI
...,...,...,...,...,...
160,old,Australia Post,1809,CAT16,AUS
161,old,Bank of New Zealand,1861,CAT3,NZL
162,old,European Trust Company,1991,CAT3,VUT
0,new,Fiji Times,1869,CAT13,FJI


In [87]:
mixed_business_country = mixed_business.merge(countries, on='country_code', how='outer', validate='one_to_one')
print(f'DataFrame size: {mixed_business_country.shape}')
pd.DataFrame({'count_null': mixed_business_country.isna().sum()})

DataFrame size: (195, 7)


,count_null
status,30
business,30
year_founded,30
category_code,30
country_code,0
country,0
continent,0


In [82]:
count_missing_business = mixed_business_country.groupby('continent')['business'].apply(lambda cont: cont.isna().sum()).reset_index()
count_missing_business.rename(columns={'business': 'missing_business'}, inplace=True)
# Count the no. of Countries with missing business data by Continent
count_missing_business

,continent,missing_business
0,Africa,3
1,Asia,7
2,Europe,2
3,North America,5
4,Oceania,10
5,South America,3


In [88]:
business_country_category = businesses.merge(countries, on='country_code', how='inner', validate='one_to_one') \
                            .merge(categories, on='category_code', how='left', validate='many_to_one')
print(f'DataFrame size: {business_country_category.shape}')
pd.DataFrame({'is_null': business_country_category.isna().any()})

DataFrame size: (163, 7)


,is_null
business,False
year_founded,False
category_code,False
country_code,False
country,False
continent,False
category,False


In [84]:
business_country_category.sort_values('year_founded', ascending=True, inplace=True)
oldest_business_category = business_country_category.groupby(['continent', 'category'])['year_founded'].min().reset_index()
oldest_business_category

,continent,category,year_founded
0,Africa,Agriculture,1947
1,Africa,Aviation & Transport,1854
2,Africa,Banking & Finance,1892
3,Africa,"Distillers, Vintners, & Breweries",1933
4,Africa,Energy,1968
5,Africa,Food & Beverages,1878
6,Africa,Manufacturing & Production,1820
7,Africa,Media,1943
8,Africa,Mining,1962
9,Africa,Postal Service,1772


In [85]:
oldest_business_by_continent_category = oldest_business_category.loc[oldest_business_category.groupby('continent')['year_founded'].idxmin()]
# List of Oldest Business Category on each Continent
oldest_business_by_continent_category

,continent,category,year_founded
9,Africa,Postal Service,1772
15,Asia,Construction,578
29,Europe,"Cafés, Restaurants & Bars",803
45,North America,Manufacturing & Production,1534
50,Oceania,Postal Service,1809
51,South America,Banking & Finance,1565
